# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [167]:
import hashlib
from itertools import combinations
import pandas as pd

class RDBMSNormalizer:
    def __init__(self, csv_file, fd_file):
        # Load input data
        try:
            self.data = pd.read_csv(csv_file)
        except FileNotFoundError:
            print(f"Error: File '{csv_file}' not found.")
            self.data = None
            
        with open('output.txt', 'w') as f:
            f.write('')
            f.close()
            
        self.primary_keys = []
        self.candidate_keys = []
        self.functional_dependencies = self.load_dependencies(fd_file)
        self.functional_dependencies_dict = self.parse_dependencies(self.functional_dependencies)
        self.normalized_tables = {}  # Store each normalized version of tables
        self.updated_functional_dependencies = {}  # Store updated functional dependencies for each table
        self.decomposition_log = []  # Track decompositions for dependencies
        self.used_primary_keys = []  # Track primary keys being used
        self.used_mvds = []  # Track MVDs being used
        
    def determine_highest_normal_form(self, table_data, primary_key, dependencies):
        """
        Determines the highest normal form (1NF, 2NF, 3NF, BCNF, 4NF) for the given table.
        Args:
            table_data (DataFrame): The input table data as a pandas DataFrame.
            primary_key (list): The primary key for the input table.
            dependencies (list): A list of functional and multi-valued dependencies for the input table.
        """
        print(f"\nChecking normal forms for the input table:")
        print(f"Primary key: {primary_key}")
        print(f"Dependencies: {dependencies}")
    
        # Separate functional dependencies and multi-valued dependencies
        functional_dependencies = []
        mvds = []
    
        for dep in dependencies:
            if '-->>' in dep:
                determinant, dependent = dep.split('-->>')
                mvds.append((determinant.strip(), dependent.strip()))
            elif '->' in dep:
                determinant, dependent = dep.split('->')
                functional_dependencies.append((determinant.strip(), dependent.strip()))
    
        # Check for 1NF
        if not self.is_in_1nf(table_data):
            return "Not in 1NF"
    
        # Check for 2NF
        if not self.is_in_2nf(functional_dependencies, primary_key):
            return "1NF"
    
        # Check for 3NF
        if not self.is_in_3nf(functional_dependencies, primary_key):
            return "2NF"
    
        # Check for BCNF
        if not self.is_in_bcnf(functional_dependencies, primary_key, table_data):
            return "3NF"
    
        # Check for 4NF
        if not self.is_in_4nf(mvds, functional_dependencies, primary_key, table_data):
            return "BCNF"
    
        return "4NF"
    
    def is_in_1nf(self, table_data):
        """
        Check if the table is in 1NF (no repeating groups or arrays).
        Args:
            table_data (DataFrame): The input table data as a pandas DataFrame.
        """
        for column in table_data.columns:
            if table_data[column].apply(lambda x: isinstance(x, list) or isinstance(x, dict) or ',' in str(x)).any():
                print(f"Repeating group found in column: {column}")
                return False
        return True
    
    def is_in_2nf(self, functional_dependencies, primary_key):
        """
        Check if the table is in 2NF (no partial dependencies).
        """
        for determinant, dependent in functional_dependencies:
            determinant_attrs = determinant.split(', ')
            if self.is_partial_dependency(determinant_attrs, primary_key):
                print(f"Partial dependency found: {determinant} -> {dependent}")
                return False
        return True
    
    def is_in_3nf(self, functional_dependencies, primary_key):
        """
        Check if the table is in 3NF (no transitive dependencies).
        """
        for determinant, dependent in functional_dependencies:
            determinant_attrs = determinant.split(', ')
            dependent_attrs = dependent.split(', ')
            if self.is_transitive_dependency(determinant_attrs, dependent_attrs, primary_key):
                print(f"Transitive dependency found: {determinant} -> {dependent}")
                return False
        return True
    
    def is_in_bcnf(self, functional_dependencies, primary_key, table_data):
        """
        Check if the table is in BCNF (every determinant is a candidate key).
        Args:
            functional_dependencies (list): A list of functional dependencies for the table.
            primary_key (list): The primary key of the table.
            table_data (DataFrame): The input table data as a pandas DataFrame.
        """
        for determinant, dependent in functional_dependencies:
            determinant_attrs = determinant.split(', ')
            if not self.is_superkey(determinant_attrs, table_data):
                print(f"Determinant {determinant} is not a superkey, violating BCNF.")
                return False
        return True
    
    def is_in_4nf(self, mvds, functional_dependencies, primary_key, table_data):
        """
        Check if the table is in 4NF (no non-trivial multivalued dependencies).
        Args:
            mvds (list): A list of multi-valued dependencies in the form of (determinant, dependent).
            functional_dependencies (list): A list of functional dependencies for the table.
            primary_key (list): The primary key of the table.
            table_data (DataFrame): The input table data as a pandas DataFrame.
        """
        for mvd in mvds:
            determinant, dependent = mvd
            determinant_attrs = determinant.split(', ')
            dependent_attrs = dependent.split(', ')
    
            # Check if the MVD is non-trivial
            if set(determinant_attrs) != set(primary_key):
                print(f"Non-trivial multivalued dependency detected: {determinant} -->> {dependent}")
                return False
    
        # Check for additional violations related to FDs and MVDs combined
        for fd in functional_dependencies:
            determinant, dependent = fd
            determinant_attrs = determinant.split(', ')
            dependent_attrs = dependent.split(', ')
            
            # If an FD violates 4NF conditions in combination with an MVD, identify the issue
            for mvd in mvds:
                mvd_determinant, mvd_dependent = mvd
                if set(mvd_determinant.split(', ')) == set(determinant_attrs) and not set(dependent_attrs).issubset(set(primary_key)):
                    print(f"Violation of 4NF found: FD '{determinant} -> {dependent}' conflicts with MVD '{mvd_determinant} -->> {mvd_dependent}'")
                    return False
    
        return True
    
    def is_partial_dependency(self, determinant_attrs, primary_key):
        """
        Determines if the given determinant is a partial dependency of the primary key.
        A partial dependency occurs if a non-prime attribute is determined by part of a composite key.
        """
        return set(determinant_attrs).issubset(set(primary_key)) and set(determinant_attrs) != set(primary_key)
    
    def is_transitive_dependency(self, determinant_attrs, dependent_attrs, primary_key):
        """
        Determines if a given dependency is a transitive dependency.
        """
        # If determinant is a subset of the primary key, it's not a transitive dependency
        if set(determinant_attrs).issubset(set(primary_key)):
            return False
    
        # Check if determinant is non-prime (not part of the primary key) and the dependent is also non-prime
        is_non_prime = not set(determinant_attrs).issubset(set(primary_key))
        has_non_key_dependents = any(attr not in primary_key for attr in dependent_attrs)
    
        return is_non_prime and has_non_key_dependents

    def identify_mvd(self, functional_dependencies):
        """
        Identifies multivalued dependencies (MVDs) from the functional dependencies.
        Args:
            functional_dependencies (list): A list of functional dependencies or multivalued dependencies.
        
        Returns:
            list: A list of multivalued dependencies in the form (determinant, dependent).
        """
        mvds = []
        for fd in functional_dependencies:
            # Check if the dependency is explicitly marked as an MVD (using -->> syntax)
            if '-->>' in fd:
                determinant, dependent = fd.split('-->>')
                mvds.append((determinant.strip(), dependent.strip().split(', ')))
        return mvds

    def load_dependencies(self, file_path):
        """
        Load functional and multivalued dependencies from the given file.
        """
        try:
            with open(file_path, 'r') as file:
                return [line.strip() for line in file.readlines()]
        except FileNotFoundError:
            print(f"Error: File '{file_path}' not found.")
            return []

    def split_multivalued_column(self, primary_keys, column_name):
        """
        Splits a multi-valued column into separate rows for normalization purposes.
        """
        if column_name not in self.data.columns:
            print(f"Error: Column '{column_name}' not found in data.")
            return self.data

        # Create new rows based on multi-valued columns
        rows = []
        for _, row in self.data.iterrows():
            values = row[column_name]
            if pd.notna(values) and isinstance(values, str):
                # Split multi-valued data by comma or other delimiters
                for value in values.split(', '):
                    new_row = row.copy()
                    new_row[column_name] = value
                    rows.append(new_row)
            else:
                rows.append(row)

        return pd.DataFrame(rows)

    def determine_minimal_candidate_keys(self, data):
        """
        Determine minimal candidate keys for the given data.
        """
        from itertools import combinations

        columns = list(data.columns)
        for r in range(1, len(columns) + 1):
            for combination in combinations(columns, r):
                if data[list(combination)].drop_duplicates().shape[0] == data.shape[0]:
                    return list(combination)
        return []

    def normalize_1nf(self):
        """
        Normalizes the data to First Normal Form (1NF) by removing multi-valued attributes.
        """
        print("Normalizing to 1NF")
        primary_keys = self.primary_keys
    
        for column in self.data.columns:
            if any(isinstance(value, str) and ', ' in str(value) for value in self.data[column]):
                # Assume multi-valued attributes contain comma-separated values
                new_table_df = self.split_multivalued_column(primary_keys, column)
    
                # Create new table with only the relevant primary keys and the multivalued column
                relevant_columns = primary_keys + [column]
                new_table_df = new_table_df[relevant_columns].drop_duplicates()
    
                # Store the new normalized table in self.normalized_tables
                table_name = f"{column}_1NF"
                self.normalized_tables[table_name] = {
                    'data': new_table_df,
                    'primary_key': primary_keys + [column]
                }
    
        # Store the base table without multivalued attributes
        non_multivalued_columns = [
            col for col in self.data.columns if not any(isinstance(value, str) and ', ' in str(value) for value in self.data[col])
        ]
        self.normalized_tables['BaseTable_1NF'] = {
            'data': self.data[non_multivalued_columns].drop_duplicates(),
            'primary_key': primary_keys
        }

    def normalize_2nf(self):
        """
        Normalizes the data to Second Normal Form (2NF).
        """
        print("Normalizing to 2NF...")
        tables_to_process = list(self.normalized_tables.keys())
    
        while tables_to_process:
            table_name = tables_to_process.pop(0)
            table_data = self.normalized_tables[table_name]['data']
            primary_key = self.normalized_tables[table_name]['primary_key']
    
            # Extract the updated functional dependencies from the current table structure
            updated_dependencies = self.extract_dependencies(table_name, table_data, primary_key, self.functional_dependencies_dict['fd'])
    
            self.updated_functional_dependencies[table_name] = updated_dependencies
            partial_dependencies = []
    
            print(f"\nProcessing table: {table_name}")
            print(f"Primary key for {table_name}: {primary_key}")
            print(f"Functional dependencies for {table_name}: {updated_dependencies}")
    
            # Iterate through each functional dependency and check for partial dependencies
            for fd in updated_dependencies:
                determinant, dependent = fd.split('->')
                determinant_attrs = determinant.strip().split(', ')
                dependent_attrs = dependent.strip().split(', ')
    
                # Log which determinant and dependent attributes are being analyzed
                print(f"Analyzing functional dependency: {determinant_attrs} -> {dependent_attrs}")
                # Handle multivalued dependencies here if applicable
                for mvd in self.functional_dependencies_dict['mvd']:
                    if set(mvd[0].split(', ')).issubset(set(primary_key)):
                        print(f"Multivalued dependency found: {mvd[0]} -->> {mvd[1]}")
    
                # Identify partial dependencies: determinant is a subset of the primary key but not the full primary key
                if self.is_partial_dependency(determinant_attrs, primary_key):
                    print(f"Partial dependency detected: {determinant_attrs} -> {dependent_attrs}")
                    partial_dependencies.append((determinant_attrs, dependent_attrs))
    
                    # Create a new table for the partial dependency
                    new_table_name = f"{'_'.join(dependent_attrs)}_2NF"
                    new_table = table_data[determinant_attrs + dependent_attrs].drop_duplicates()
    
                    # Store the new table with the partial dependency removed
                    self.normalized_tables[new_table_name] = {
                        'data': new_table,
                        'primary_key': determinant_attrs
                    }
    
                    # Remove the dependent attributes from the original table
                    table_data = table_data.drop(columns=dependent_attrs)
                    
                    # Update functional dependencies for the new table
                    new_fd = f"{', '.join(determinant_attrs)} -> {', '.join(dependent_attrs)}"
                    self.updated_functional_dependencies[new_table_name] = [new_fd]
                    tables_to_process.append(new_table_name)
                    print(f"Partial Dependency found and decomposed: {new_fd}")
    
            # Store the updated base table after removing partial dependencies
            self.normalized_tables[table_name] = {
                'data': table_data,
                'primary_key': primary_key
            }
    
            # Update functional dependencies for the base table
            remaining_dependencies = [
                fd for fd in updated_dependencies if fd not in [
                    f"{', '.join(d)} -> {', '.join(dep)}" for d, dep in partial_dependencies
                ]
            ]
            self.updated_functional_dependencies[table_name] = remaining_dependencies
    
            # Check if the table is already in 2NF
            if not partial_dependencies:
                print(f"Table {table_name} is already in 2NF.")
            else:
                print(f"Table {table_name} has been decomposed to remove partial dependencies. Partial dependencies found: {partial_dependencies}")

    def normalize_3nf(self):
        """
        Normalizes the data to Third Normal Form (3NF) by removing transitive dependencies.
        """
        print("Normalizing to 3NF...")
        tables_to_process = list(self.normalized_tables.keys())

        while tables_to_process:
            table_name = tables_to_process.pop(0)
            table_data = self.normalized_tables[table_name]['data']
            primary_key = self.normalized_tables[table_name]['primary_key']

            updated_dependencies = self.extract_dependencies(table_name, table_data, primary_key, self.functional_dependencies_dict['fd'])
            self.updated_functional_dependencies[table_name] = updated_dependencies
            transitive_dependencies = []

            print(f"\nProcessing table: {table_name}")
            print(f"Primary key for {table_name}: {primary_key}")
            print(f"Functional dependencies for {table_name}: {updated_dependencies}")

            for fd in updated_dependencies:
                determinant, dependent = fd.split('->')
                determinant_attrs = determinant.strip().split(', ')
                dependent_attrs = dependent.strip().split(', ')

                print(f"Analyzing functional dependency: {determinant_attrs} -> {dependent_attrs}")

                if self.is_transitive_dependency(determinant_attrs, dependent_attrs, primary_key):
                    print(f"Transitive dependency detected: {determinant_attrs} -> {dependent_attrs}")
                    transitive_dependencies.append((determinant_attrs, dependent_attrs))

                    # Create a new table for the transitive dependency
                    new_table_name = f"{'_'.join(dependent_attrs)}_3NF"
                    new_table = table_data[determinant_attrs + dependent_attrs].drop_duplicates()

                    self.normalized_tables[new_table_name] = {
                        'data': new_table,
                        'primary_key': determinant_attrs
                    }

                    table_data = table_data.drop(columns=dependent_attrs)

                    new_fd = f"{', '.join(determinant_attrs)} -> {', '.join(dependent_attrs)}"
                    self.updated_functional_dependencies[new_table_name] = [new_fd]
                    tables_to_process.append(new_table_name)
                    print(f"Transitive Dependency found and decomposed: {new_fd}")

            self.normalized_tables[table_name] = {
                'data': table_data,
                'primary_key': primary_key
            }

            remaining_dependencies = [
                fd for fd in updated_dependencies if fd not in [
                    f"{', '.join(d)} -> {', '.join(dep)}" for d, dep in transitive_dependencies
                ]
            ]
            self.updated_functional_dependencies[table_name] = remaining_dependencies

            if not transitive_dependencies:
                print(f"Table {table_name} is already in 3NF.")
            else:
                print(f"Table {table_name} has been decomposed to remove transitive dependencies. Transitive dependencies found: {transitive_dependencies}")
    
    def normalize_bcnf(self):
        """
        Normalizes the data to Boyce-Codd Normal Form (BCNF).
        """
        print("Normalizing to BCNF...")
        tables_to_process = list(self.normalized_tables.keys())

        while tables_to_process:
            table_name = tables_to_process.pop(0)
            table_data = self.normalized_tables[table_name]['data']
            primary_key = self.normalized_tables[table_name]['primary_key']

            updated_dependencies = self.extract_dependencies(table_name, table_data, primary_key, self.functional_dependencies_dict['fd'])
            self.updated_functional_dependencies[table_name] = updated_dependencies
            bcnf_violations = []

            print(f"\nProcessing table: {table_name}")
            print(f"Primary key for {table_name}: {primary_key}")
            print(f"Functional dependencies for {table_name}: {updated_dependencies}")

            # Identify BCNF violations: functional dependencies where the determinant is not a superkey
            for fd in updated_dependencies:
                determinant, dependent = fd.split('->')
                determinant_attrs = determinant.strip().split(', ')
                dependent_attrs = dependent.strip().split(', ')

                print(f"Analyzing functional dependency: {determinant_attrs} -> {dependent_attrs}")

                if not self.is_superkey(determinant_attrs, table_data):
                    print(f"BCNF violation detected: {determinant_attrs} -> {dependent_attrs}")
                    bcnf_violations.append((determinant_attrs, dependent_attrs))

                    # Create a new table for the violating dependency
                    new_table_name = f"{'_'.join(dependent_attrs)}_BCNF"
                    new_table = table_data[determinant_attrs + dependent_attrs].drop_duplicates()

                    # Store the new table with the violating dependency removed
                    self.normalized_tables[new_table_name] = {
                        'data': new_table,
                        'primary_key': determinant_attrs
                    }

                    # Remove the dependent attributes from the original table
                    table_data = table_data.drop(columns=dependent_attrs)

                    # Update functional dependencies for the new table
                    new_fd = f"{', '.join(determinant_attrs)} -> {', '.join(dependent_attrs)}"
                    self.updated_functional_dependencies[new_table_name] = [new_fd]
                    tables_to_process.append(new_table_name)
                    print(f"BCNF violation found and decomposed: {new_fd}")

            # Store the updated base table after removing BCNF violations
            self.normalized_tables[table_name] = {
                'data': table_data,
                'primary_key': primary_key
            }

            # Update functional dependencies for the base table
            remaining_dependencies = [
                fd for fd in updated_dependencies if fd not in [
                    f"{', '.join(d)} -> {', '.join(dep)}" for d, dep in bcnf_violations
                ]
            ]
            self.updated_functional_dependencies[table_name] = remaining_dependencies

            if not bcnf_violations:
                print(f"Table {table_name} is already in BCNF.")
            else:
                print(f"Table {table_name} has been decomposed to remove BCNF violations. Violations found: {bcnf_violations}")
    
    def normalize_4nf(self):
        """
        Normalize the table to 4NF by decomposing it based on multivalued dependencies (MVDs).
        Create separate relations for each MVD until no non-trivial MVDs remain.
        """
        print("Normalizing to 4NF...")
        tables_to_process = [{'data': self.data, 'table_name': 'BaseTable'}]
        decomposed_tables = []
    
        seen_hashes = set()
    
        while tables_to_process:
            table = tables_to_process.pop(0)
            table_data = table['data']
            table_name = table['table_name']
    
            # Generate a hash for the current table to track it
            table_hash = hashlib.md5(pd.util.hash_pandas_object(table_data, index=True).values).hexdigest()
    
            # If the table has already been processed, skip it
            if table_hash in seen_hashes:
                continue
            seen_hashes.add(table_hash)
    
            # Identify MVDs in the current table
            mvds = self.identify_multivalued_dependencies(table_data)
    
            if not mvds:
                # No MVDs found, add the table to final decomposed tables
                self.normalized_tables[table_name] = {
                    'data': table_data,
                    'primary_key': self.determine_primary_key(table_data)
                }
                decomposed_tables.append(table_data)
                continue
    
            # Decompose based on the identified MVDs
            for mvd in mvds:
                determinant, dependent = mvd
    
                # Ensure determinant and dependent are treated as lists
                if isinstance(determinant, str):
                    determinant = [determinant]
                if isinstance(dependent, str):
                    dependent = [dependent]
    
                print(f"Decomposing table {table_name} based on MVD: {determinant} -->> {dependent}")
    
                # Decompose the table into two relations as per the MVD
                relation_1 = table_data[determinant + dependent].drop_duplicates().reset_index(drop=True)
                relation_2 = table_data[determinant + [col for col in table_data.columns if col not in dependent]].drop_duplicates().reset_index(drop=True)
    
                # Assign names to the new relations
                relation_1_name = f"{table_name}_{'_'.join(determinant)}_dependent_4NF"
                relation_2_name = f"{table_name}_{'_'.join(determinant)}_remaining_4NF"
    
                # Track the new decomposed tables if they are unique
                for relation, name in zip([relation_1, relation_2], [relation_1_name, relation_2_name]):
                    relation_hash = hashlib.md5(pd.util.hash_pandas_object(relation, index=True).values).hexdigest()
                    if relation_hash not in seen_hashes:
                        seen_hashes.add(relation_hash)
                        self.normalized_tables[name] = {
                            'data': relation,
                            'primary_key': self.determine_primary_key(relation)
                        }
                        decomposed_tables.append(relation)
                        tables_to_process.append({'data': relation, 'table_name': name})
    
        return decomposed_tables




    def identify_multivalued_dependencies(self, table_data):
        """
        Identifies non-trivial multivalued dependencies in the table.
        Args:
            table_data (DataFrame): The current table data as a pandas DataFrame.
        Returns:
            list: A list of multivalued dependencies that exist in the table.
        """
        mvds = []
    
        for mvd in self.functional_dependencies_dict['mvd']:
            determinant, dependent = mvd
            determinant = determinant.split(", ")
            dependent = dependent.split(", ")
    
            # Check if the determinant and dependent columns exist in the table data
            if not all(col in table_data.columns for col in determinant + dependent):
                continue  # Skip this MVD if columns are missing
    
            # Check if the determinant values are independent of the dependent values
            try:
                if not table_data.groupby(determinant)[dependent].nunique().eq(1).all().all():
                    mvds.append(mvd)
            except KeyError as e:
                print(f"Warning: {e}")
                continue
    
        return mvds
    
    def decompose_based_on_mvd(self, table_data, determinant, dependent):
        """
        Decomposes the table based on the given multivalued dependency.
        Args:
            table_data (DataFrame): The current table data as a pandas DataFrame.
            determinant (list): The determinant attributes of the MVD.
            dependent (list): The dependent attributes of the MVD.
        Returns:
            list: A list of DataFrames resulting from the decomposition.
        """
        # Ensure determinant and dependent are lists
        if isinstance(determinant, str):
            determinant = [determinant]
        if isinstance(dependent, str):
            dependent = [dependent]
    
        remaining_columns = [col for col in table_data.columns if col not in dependent]
    
        # Create new relations based on decomposition
        try:
            relation_1 = table_data[determinant + dependent].drop_duplicates().reset_index(drop=True)
            relation_2 = table_data[remaining_columns].drop_duplicates().reset_index(drop=True)
        except KeyError as e:
            print(f"Error: {e}")
            return [table_data]
    
        return [relation_1, relation_2]


    def determine_primary_key(self, table_data):
        """
        Determine minimal candidate keys for the given data.
        Args:
            table_data (DataFrame): The data for which to determine the primary key.
        Returns:
            list: A list of columns that form the primary key.
        """
        from itertools import combinations

        columns = list(table_data.columns)
        for r in range(1, len(columns) + 1):
            for combination in combinations(columns, r):
                if table_data[list(combination)].drop_duplicates().shape[0] == table_data.shape[0]:
                    return list(combination)
        return []

    def identify_remaining_mvds(self, table_data, determinant_attrs):
        """
        Identifies any remaining MVDs in the table after decomposition.
        Args:
            table_data (DataFrame): The current table data as a pandas DataFrame.
            determinant_attrs (list): The determinant attributes of the MVD.
        
        Returns:
            list: A list of remaining MVDs in the form (determinant, dependent).
        """
        remaining_mvds = []
        for mvd in self.functional_dependencies_dict['mvd']:
            determinant, dependent = mvd
            if set(determinant.split(', ')).issubset(set(table_data.columns)):
                remaining_mvds.append(mvd)
        return remaining_mvds
    
    def normalize_5nf(self):
        print("Normalizing to 5NF...")
        tables_to_process = [{'data': self.data, 'table_name': 'BaseTable'}]
        decomposed_tables = []

        while tables_to_process:
            table = tables_to_process.pop(0)
            table_data = table['data']
            table_name = table['table_name']

            join_dependencies = self.identify_join_dependencies(table_data)
            while join_dependencies:
                jd = join_dependencies.pop(0)
                print(f"Decomposing table {table_name} based on join dependency: {jd}")
                relations = self.decompose_based_on_join_dependency(table_data, jd)

                for i, relation in enumerate(relations):
                    new_table_name = f"{table_name}_JD_{i + 1}_5NF"
                    self.normalized_tables[new_table_name] = {
                        'data': relation,
                        'primary_key': self.determine_primary_key(relation)
                    }
                    decomposed_tables.append(relation)
                    tables_to_process.append({'data': relation, 'table_name': new_table_name})

            if not join_dependencies:
                self.normalized_tables[table_name] = {
                    'data': table_data,
                    'primary_key': self.determine_primary_key(table_data)
                }
                decomposed_tables.append(table_data)

        return decomposed_tables

    def identify_join_dependencies(self, table_data):
        """
        Identifies non-trivial join dependencies in the table.
        Args:
            table_data (DataFrame): The current table data as a pandas DataFrame.
        Returns:
            list: A list of join dependencies that exist in the table.
        """
        join_dependencies = []

        columns = list(table_data.columns)
        for r in range(1, len(columns)):
            for combination in combinations(columns, r):
                remaining_columns = [col for col in columns if col not in combination]

                # Check if splitting into two sets yields tables that can be naturally joined to reconstruct the original table
                left_df = table_data[list(combination)].drop_duplicates()
                right_df = table_data[remaining_columns].drop_duplicates()

                # Perform a natural join on the split tables
                reconstructed_df = pd.merge(left_df, right_df, how='inner')

                # If the join yields the same number of rows as the original table, we have a join dependency
                if reconstructed_df.shape[0] == table_data.shape[0]:
                    join_dependencies.append((list(combination), remaining_columns))

        return join_dependencies

    def decompose_based_on_join_dependency(self, table_data, join_dependency):
        """
        Decomposes the table based on the given join dependency.
        Args:
            table_data (DataFrame): The current table data as a pandas DataFrame.
            join_dependency: The join dependency to decompose on.
        Returns:
            list: A list of DataFrames resulting from the decomposition.
        """
        left_columns, right_columns = join_dependency
        left_table = table_data[left_columns].drop_duplicates().reset_index(drop=True)
        right_table = table_data[right_columns].drop_duplicates().reset_index(drop=True)
        return [left_table, right_table]
    
    def is_superkey(self, determinant_attrs, table_data):
        """
        Helper function to check if the given determinant attributes form a superkey.
        A superkey uniquely identifies each row in the table.
        """
        print(f"Checking if {determinant_attrs} is a superkey.")
        # If determinant attributes uniquely identify rows in the table, it is a superkey
        if table_data[determinant_attrs].drop_duplicates().shape[0] == table_data.shape[0]:
            print(f"{determinant_attrs} is a superkey.")
            return True
        print(f"{determinant_attrs} is not a superkey.")
        return False

    def parse_dependencies(self, dependencies):
        """
        Parse the loaded dependencies to separate functional and multivalued dependencies.
        """
        fd_dict = {'fd': [], 'mvd': []}
        for dep in dependencies:
            if '-->>' in dep:
                determinant, dependent = dep.split('-->>')
                fd_dict['mvd'].append((determinant.strip().replace('{', '').replace('}', ''),
                                       dependent.strip().replace('{', '').replace('}', '')))
            elif '-->' in dep:
                determinant, dependent = dep.split('-->')
                fd_dict['fd'].append((determinant.strip().replace('{', '').replace('}', ''),
                                      dependent.strip().replace('{', '').replace('}', '')))
        return fd_dict

    def extract_dependencies(self, table_name, table_data, primary_key, functional_dependencies):
        """
        Extract functional dependencies for the given table based on the provided dependencies.
        Enhanced to check if the determinant uniquely identifies the dependent attributes.
        """
        extracted_dependencies = []
        for determinant, dependent in functional_dependencies:
            determinant_attrs = determinant.split(', ')
            dependent_attrs = dependent.split(', ')

            # Validate that all determinant and dependent attributes exist in the table columns
            if all(attr in table_data.columns for attr in determinant_attrs + dependent_attrs):
                # Check if determinant attributes are present in the table
                if set(determinant_attrs).issubset(set(table_data.columns)):
                    # Check if determinant uniquely identifies the dependent
                    if table_data[determinant_attrs].drop_duplicates().shape[0] == table_data.drop_duplicates(subset=determinant_attrs + dependent_attrs).shape[0]:
                        extracted_dependencies.append(f"{determinant} -> {dependent}")
            # else:
                # print(f"Warning: Dependency '{determinant} -> {dependent}' contains attributes not in table '{table_name}'")
        return extracted_dependencies

    def is_partial_dependency(self, determinant_attrs, primary_key):
        """
        Helper function to check if the given determinant attributes form a partial dependency.
        A partial dependency exists if:
        1. The determinant is a proper subset of the primary key, or
        2. The determinant is a proper subset of any candidate key.
        """
        print(f"Checking if {determinant_attrs} is a partial dependency of primary key {primary_key}")
    
        # Check if the determinant is a proper subset of the primary key
        if set(determinant_attrs).issubset(set(primary_key)) and set(determinant_attrs) != set(primary_key):
            print(f"{determinant_attrs} is a proper subset of {primary_key}. This is a partial dependency.")
            return True
    
        # Check for candidate keys
        for candidate_key in self.candidate_keys:
            print(f"Checking if {determinant_attrs} is a partial dependency of candidate key {candidate_key}")
            if set(determinant_attrs).issubset(set(candidate_key)) and set(determinant_attrs) != set(candidate_key):
                print(f"{determinant_attrs} is a proper subset of candidate key {candidate_key}. This is a partial dependency.")
                return True
    
        return False

    def is_transitive_dependency(self, determinant_attrs, dependent_attrs, primary_key):
        """
        Helper function to check if the given functional dependency is a transitive dependency.
        A transitive dependency exists if:
        1. The determinant is not a subset of the primary key, and
        2. The dependent is also not part of the primary key.
        """
        print(f"Checking if {determinant_attrs} -> {dependent_attrs} is a transitive dependency with respect to primary key {primary_key}")
        if not set(determinant_attrs).issubset(set(primary_key)) and not set(dependent_attrs).issubset(set(primary_key)):
            print(f"{determinant_attrs} -> {dependent_attrs} is a transitive dependency.")
            return True
        return False

    def generate_sql(self):
        """
        Generates optimized SQL statements for creating normalized tables.
        """
        print("Generating optimized SQL for normalized tables...")
        for table_name, table_info in self.normalized_tables.items():
            table_data = table_info['data']
            primary_key = table_info['primary_key']

            # Automatically determine appropriate SQL data types based on data analysis
            columns = []
            for col in table_data.columns:
                col_data = table_data[col]
                col_dtype = col_data.dtype
                
                if pd.api.types.is_integer_dtype(col_dtype):
                    data_type = "INT"
                elif pd.api.types.is_float_dtype(col_dtype):
                    data_type = "FLOAT"
                elif pd.api.types.is_datetime64_any_dtype(col_dtype):
                    data_type = "DATETIME"
                elif pd.api.types.is_bool_dtype(col_dtype):
                    data_type = "BOOLEAN"
                else:
                    max_length = col_data.dropna().astype(str).str.len().max()
                    data_type = f"VARCHAR({max_length if max_length is not None else 255})"
                
                columns.append(f"{col} {data_type}")

            primary_key_str = ', '.join(primary_key) if primary_key else None
            if primary_key_str:
                create_table_sql = f"CREATE TABLE {table_name} ({', '.join(columns)}, PRIMARY KEY ({primary_key_str}));"
            else:
                create_table_sql = f"CREATE TABLE {table_name} ({', '.join(columns)});"
            
            print(create_table_sql)
            with open('output.txt', 'a') as f:
                f.write(create_table_sql + '\n')
                
    def user_input(self):
        # Step 1: Determine the highest normal form of the given input table
        print("\nDetermine the highest normal form of the input table.")
        choice = input("Would you like to determine the highest normal form of the given input table? (yes/no): ").lower()
        
        if choice == 'yes':
            highest_nf = self.determine_highest_normal_form(self.data, self.primary_keys, self.functional_dependencies)
            print(f"\nThe input table currently satisfies up to: {highest_nf}")
    
        # Step 2: Ask user to proceed with normalization
        while True:
            # print("\nChoose the highest normal form to normalize the data: 1.1NF 2.NF 3.3NF 4.BCNF 5.4NF 6.5NF")
            choice = input("Choose the highest normal form to normalize the data: 1.1NF 2.NF 3.3NF 4.BCNF 5.4NF 6.5NF\n")
    
            # Perform normalization based on the user's choice
            if choice == '1':
                self.normalize_1nf()
            elif choice == '2':
                self.normalize_1nf()
                self.normalize_2nf()
            elif choice == '3':
                self.normalize_1nf()
                self.normalize_2nf()
                self.normalize_3nf()
            elif choice == '4':
                self.normalize_1nf()
                self.normalize_2nf()
                self.normalize_3nf()
                self.normalize_bcnf()
            elif choice == '5':
                self.normalize_1nf()
                self.normalize_2nf()
                self.normalize_3nf()
                self.normalize_bcnf()
                self.normalize_4nf()
            elif choice == '6':
                self.normalize_1nf()
                self.normalize_2nf()
                self.normalize_3nf()
                self.normalize_bcnf()
                self.normalize_4nf()
                self.normalize_5nf()
            else:
                print("Invalid choice. Please try again.")
                continue
    
            break
            
if __name__ == "__main__":
    # Prompt user for the primary and candidate keys directly
    primary_keys = [key.strip() for key in input("Enter the primary keys (comma-separated): ").split(',')]
    candidate_keys = [key.strip() for key in input("Enter the candidate keys (comma-separated, or press Enter if none): ").split(',')] if input("Are there any candidate keys? (yes/no): ").lower() == 'yes' else []

    normalizer = RDBMSNormalizer(
        csv_file='inputTable2.csv',
        fd_file='fd2.txt'
    )
    normalizer.primary_keys = primary_keys
    normalizer.candidate_keys = candidate_keys
    normalizer.user_input()
    normalizer.generate_sql()



Determine the highest normal form of the input table.

Checking normal forms for the input table:
Primary key: ['OrderID', 'DrinkID', 'FoodID', 'DrinkAllergen']
Dependencies: ['OrderID -->> DrinkID', 'OrderID -->> FoodID']
Non-trivial multivalued dependency detected: OrderID -->> DrinkID

The input table currently satisfies up to: BCNF
Normalizing to 1NF
Normalizing to 2NF...

Processing table: BaseTable_1NF
Primary key for BaseTable_1NF: ['OrderID', 'DrinkID', 'FoodID', 'DrinkAllergen']
Functional dependencies for BaseTable_1NF: []
Table BaseTable_1NF is already in 2NF.
Normalizing to 3NF...

Processing table: BaseTable_1NF
Primary key for BaseTable_1NF: ['OrderID', 'DrinkID', 'FoodID', 'DrinkAllergen']
Functional dependencies for BaseTable_1NF: []
Table BaseTable_1NF is already in 3NF.
Normalizing to BCNF...

Processing table: BaseTable_1NF
Primary key for BaseTable_1NF: ['OrderID', 'DrinkID', 'FoodID', 'DrinkAllergen']
Functional dependencies for BaseTable_1NF: []
Table BaseTable_1

AttributeError: 'DataFrame' object has no attribute 'dtype'